Kesci-提供银行精准解决方案-model

In [24]:
!pip install tqdm

In [55]:
#导入相应的包
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score,precision_score
from tqdm import tqdm_notebook
from tqdm import tqdm
from sklearn.model_selection import  StratifiedKFold,KFold,RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import time
import datetime
import warnings
warnings.filterwarnings("ignore")

In [8]:
#导入数据
train=pd.read_csv('C:\\Users\\Think\\Desktop\\input\\train_set.csv')
test=pd.read_csv('C:/Users/Think/Desktop/input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
ID           25317 non-null int64
age          25317 non-null int64
job          25317 non-null object
marital      25317 non-null object
education    25317 non-null object
default      25317 non-null object
balance      25317 non-null int64
housing      25317 non-null object
loan         25317 non-null object
contact      25317 non-null object
day          25317 non-null int64
month        25317 non-null object
duration     25317 non-null int64
campaign     25317 non-null int64
pdays        25317 non-null int64
previous     25317 non-null int64
poutcome     25317 non-null object
y            25317 non-null int64
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
ID           10852 non-null int64
age          10852 non-null int64
job          10852 non-null object
mar

In [9]:
train.describe()#describe()函数只是统计数值特征的情况

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [10]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [11]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [16]:
dict=train.job.value_counts()
print(dict,len(dict))

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64 12


In [13]:
train.duration.value_counts()

124     119
90      110
111     104
96      101
82      101
89      100
106      99
73       99
72       98
81       97
150      96
88       95
121      95
139      95
92       95
101      95
112      95
100      94
123      94
122      94
113      93
103      92
144      92
128      92
98       92
136      92
94       91
110      91
119      91
104      90
       ... 
1528      1
1448      1
1368      1
1240      1
1176      1
1112      1
1096      1
1129      1
1145      1
1161      1
1266      1
1090      1
1273      1
1170      1
1186      1
1202      1
1218      1
1234      1
1250      1
1282      1
1970      1
1298      1
1314      1
1346      1
1490      1
1554      1
1650      1
1730      1
1842      1
2015      1
Name: duration, Length: 1388, dtype: int64

In [18]:
test['y']=-1 #test添加label列
print(len(test.columns))

18


In [41]:
data=train.append(test).reset_index(drop=True)#train集与test集合并
#data=pd.concat([train,test]).reset_index(drop=True)
data.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,36169.00000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000
mean,18085.00000,40.966961,1368.237026,15.788742,257.574497,2.771545,40.160552,0.578147,-0.218170
std,10441.23528,10.639679,3098.876172,8.314523,255.040668,3.114445,100.005886,2.365582,0.578198
min,1.00000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,-1.000000
25%,9043.00000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,-1.000000
50%,18085.00000,39.000000,449.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,27127.00000,48.000000,1437.000000,21.000000,318.000000,3.000000,-1.000000,0.000000,0.000000
max,36169.00000,95.000000,102127.000000,31.000000,3881.000000,58.000000,871.000000,275.000000,1.000000


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36169 entries, 0 to 36168
Data columns (total 27 columns):
ID                 36169 non-null int64
age                36169 non-null int64
job                36169 non-null int64
marital            36169 non-null int64
education          36169 non-null int64
default            36169 non-null int64
balance            36169 non-null int64
housing            36169 non-null int64
loan               36169 non-null int64
contact            36169 non-null int64
day                36169 non-null int64
month              36169 non-null int64
duration           36169 non-null int64
campaign           36169 non-null int64
pdays              36169 non-null int64
previous           36169 non-null int64
poutcome           36169 non-null int64
y                  36169 non-null int64
count_job          36169 non-null int64
count_marital      36169 non-null int64
count_education    36169 non-null int64
count_default      36169 non-null int64
count_housi

In [68]:
data.drop(['count_job','count_marital','count_education','count_default','count_housing','count_loan','count_contact','count_month','count_poutcome'],axis=1)

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,4,1,2,0,291,1,0,2,9,8,150,2,-1,0,3,0
1,2,42,9,0,0,0,5076,1,0,0,7,0,99,1,251,2,1,0
2,3,47,0,1,1,0,104,1,1,0,14,5,77,2,-1,0,3,0
3,4,28,4,2,1,0,-994,1,1,0,18,5,174,2,-1,0,3,0
4,5,42,9,0,1,0,2974,1,0,2,21,8,187,5,-1,0,3,0
5,6,56,7,0,1,0,450,0,0,2,9,6,47,1,-1,0,3,0
6,7,32,4,2,2,0,1716,0,0,0,21,9,157,4,-1,0,3,0
7,8,33,4,2,2,0,-359,0,0,1,29,1,59,2,-1,0,3,0
8,9,32,9,2,1,0,309,1,1,0,25,1,99,3,-1,0,3,0
9,10,55,4,1,2,0,4777,0,1,2,20,6,12,3,-1,0,3,0


In [70]:
cat_col=['job','marital','education','default','housing','loan','contact','month','poutcome']
for i in tqdm_notebook(cat_col):
    #lbl=LabelEncoder()
    data['count_'+ i]=data.groupby([i])[i].transform('count')
    #data[i]=lbl.fit_transform(data[i].astype(str))

In [71]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [72]:
feats=[i for i in data.columns if i not in ['ID','y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_job',
 'count_marital',
 'count_education',
 'count_default',
 'count_housing',
 'count_loan',
 'count_contact',
 'count_month',
 'count_poutcome']

In [77]:
model=lgb.LGBMClassifier(boosting_type="gbdt",num_leaves=30,reg_alpha=0,reglambda=0,max_depth=-1,n_estimators=1500
                        ,objective='binary',metric='auc',subsample=0.95,colsample_bytree=0.7,subsample_freq=1,
                        learning_rate=0.02,random_state=2017)
train_x=data[data['y']!=-1][feats]
train_y=data[data['y']!=-1]['y']
test_x=data[data['y']==-1][feats]
model.fit(train_x,train_y)
test_pre=model.predict_proba(test_x)[:,1]
len(test_pre)

10852

In [80]:
pre=data[data['y']==-1][['ID']]
pre['pred']=test_pre

In [81]:
pre.head()

,ID,pred
25317,25318,0.053728
25318,25319,0.003968
25319,25320,0.001854
25320,25321,0.785342
25321,25322,0.013772


In [82]:
pre.to_csv("result.csv",index=False)

In [85]:
from sklearn.model_selection import KFold
n_splits=10
kflod=KFold(n_splits=10,shuffle=True,random_state=42)
train_x=data[data['y']!=-1][feats]
train_y=data[data['y']!=-1]['y']
test_x=data[data['y']==-1][feats]
res=data[data['y']==-1][["ID"]]
res['pred']=0
for train_idx,val_idx in kflod.split(train_x):
    model.random_state=model.random_state+1
    train_x1=train_x.loc[train_idx]
    train_y1=train_y.loc[train_idx]
    test_x1=train_x.loc[val_idx]
    test_y1=train_y.loc[val_idx]
    model.fit(train_x1,train_y1,eval_set=[(train_x1,train_y1),(test_x1,test_y1)],eval_metric='auc',
             early_stopping_rounds=100)
    res['pred']+=model.predict_proba(test_x)[:,1]
res['pred']=res['pred']/10
res.to_csv('C:/Users/Think/Desktop/result1.csv',index=False)

[1]	training's auc: 0.772529	valid_1's auc: 0.773362
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.909463	valid_1's auc: 0.898501
[3]	training's auc: 0.909273	valid_1's auc: 0.896971
[4]	training's auc: 0.902155	valid_1's auc: 0.889149
[5]	training's auc: 0.917321	valid_1's auc: 0.904155
[6]	training's auc: 0.922123	valid_1's auc: 0.911541
[7]	training's auc: 0.922682	valid_1's auc: 0.911068
[8]	training's auc: 0.924879	valid_1's auc: 0.915325
[9]	training's auc: 0.924143	valid_1's auc: 0.913565
[10]	training's auc: 0.923176	valid_1's auc: 0.912257
[11]	training's auc: 0.926464	valid_1's auc: 0.916286
[12]	training's auc: 0.928391	valid_1's auc: 0.917363
[13]	training's auc: 0.929061	valid_1's auc: 0.917781
[14]	training's auc: 0.92872	valid_1's auc: 0.917038
[15]	training's auc: 0.929433	valid_1's auc: 0.917501
[16]	training's auc: 0.928812	valid_1's auc: 0.916224
[17]	training's auc: 0.929785	valid_1's auc: 0.917348
[18]	training's auc: 0.930617	

[159]	training's auc: 0.950525	valid_1's auc: 0.931476
[160]	training's auc: 0.950586	valid_1's auc: 0.931532
[161]	training's auc: 0.950678	valid_1's auc: 0.931535
[162]	training's auc: 0.950719	valid_1's auc: 0.931508
[163]	training's auc: 0.950818	valid_1's auc: 0.931514
[164]	training's auc: 0.950926	valid_1's auc: 0.931514
[165]	training's auc: 0.951032	valid_1's auc: 0.931495
[166]	training's auc: 0.951121	valid_1's auc: 0.931467
[167]	training's auc: 0.951164	valid_1's auc: 0.93147
[168]	training's auc: 0.951245	valid_1's auc: 0.931445
[169]	training's auc: 0.951378	valid_1's auc: 0.931532
[170]	training's auc: 0.951456	valid_1's auc: 0.93147
[171]	training's auc: 0.951643	valid_1's auc: 0.931759
[172]	training's auc: 0.951701	valid_1's auc: 0.931788
[173]	training's auc: 0.951749	valid_1's auc: 0.931754
[174]	training's auc: 0.951821	valid_1's auc: 0.931788
[175]	training's auc: 0.951901	valid_1's auc: 0.931789
[176]	training's auc: 0.952011	valid_1's auc: 0.931863
[177]	traini

[323]	training's auc: 0.962906	valid_1's auc: 0.933762
[324]	training's auc: 0.96297	valid_1's auc: 0.933702
[325]	training's auc: 0.963019	valid_1's auc: 0.933678
[326]	training's auc: 0.963086	valid_1's auc: 0.933646
[327]	training's auc: 0.963139	valid_1's auc: 0.933618
[328]	training's auc: 0.963227	valid_1's auc: 0.933706
[329]	training's auc: 0.963277	valid_1's auc: 0.933762
[330]	training's auc: 0.963338	valid_1's auc: 0.933783
[331]	training's auc: 0.963392	valid_1's auc: 0.933756
[332]	training's auc: 0.963441	valid_1's auc: 0.933737
[333]	training's auc: 0.963512	valid_1's auc: 0.93372
[334]	training's auc: 0.963564	valid_1's auc: 0.933695
[335]	training's auc: 0.963649	valid_1's auc: 0.933693
[336]	training's auc: 0.963709	valid_1's auc: 0.933731
[337]	training's auc: 0.963765	valid_1's auc: 0.93379
[338]	training's auc: 0.963846	valid_1's auc: 0.933734
[339]	training's auc: 0.9639	valid_1's auc: 0.933696
[340]	training's auc: 0.963946	valid_1's auc: 0.933618
[341]	training'

[480]	training's auc: 0.971396	valid_1's auc: 0.934251
[481]	training's auc: 0.971454	valid_1's auc: 0.934266
[482]	training's auc: 0.971493	valid_1's auc: 0.934291
[483]	training's auc: 0.971525	valid_1's auc: 0.934288
[484]	training's auc: 0.971565	valid_1's auc: 0.934291
[485]	training's auc: 0.971607	valid_1's auc: 0.934289
[486]	training's auc: 0.971677	valid_1's auc: 0.934282
[487]	training's auc: 0.971724	valid_1's auc: 0.934285
[488]	training's auc: 0.971764	valid_1's auc: 0.934308
[489]	training's auc: 0.971814	valid_1's auc: 0.934342
[490]	training's auc: 0.971851	valid_1's auc: 0.934345
[491]	training's auc: 0.971908	valid_1's auc: 0.934369
[492]	training's auc: 0.971942	valid_1's auc: 0.934342
[493]	training's auc: 0.97198	valid_1's auc: 0.934364
[494]	training's auc: 0.972033	valid_1's auc: 0.934389
[495]	training's auc: 0.972077	valid_1's auc: 0.934398
[496]	training's auc: 0.972124	valid_1's auc: 0.934459
[497]	training's auc: 0.972166	valid_1's auc: 0.934463
[498]	train

[658]	training's auc: 0.978508	valid_1's auc: 0.934522
[659]	training's auc: 0.978539	valid_1's auc: 0.934519
[660]	training's auc: 0.978575	valid_1's auc: 0.93454
[661]	training's auc: 0.978615	valid_1's auc: 0.934543
[662]	training's auc: 0.978646	valid_1's auc: 0.934523
[663]	training's auc: 0.978688	valid_1's auc: 0.934495
[664]	training's auc: 0.978722	valid_1's auc: 0.93452
[665]	training's auc: 0.97875	valid_1's auc: 0.934525
[666]	training's auc: 0.978788	valid_1's auc: 0.934532
[667]	training's auc: 0.978844	valid_1's auc: 0.934537
[668]	training's auc: 0.978879	valid_1's auc: 0.934565
[669]	training's auc: 0.978911	valid_1's auc: 0.934573
[670]	training's auc: 0.978939	valid_1's auc: 0.93455
[671]	training's auc: 0.978964	valid_1's auc: 0.934559
[672]	training's auc: 0.978991	valid_1's auc: 0.934528
[673]	training's auc: 0.979037	valid_1's auc: 0.934512
[674]	training's auc: 0.979079	valid_1's auc: 0.934519
[675]	training's auc: 0.979112	valid_1's auc: 0.934519
[676]	training

...

[906]	training's auc: 0.98504	valid_1's auc: 0.939289
[907]	training's auc: 0.985071	valid_1's auc: 0.939275
[908]	training's auc: 0.985088	valid_1's auc: 0.939271
[909]	training's auc: 0.985102	valid_1's auc: 0.939277
[910]	training's auc: 0.985127	valid_1's auc: 0.939296
[911]	training's auc: 0.985144	valid_1's auc: 0.939299
[912]	training's auc: 0.985179	valid_1's auc: 0.939303
[913]	training's auc: 0.985192	valid_1's auc: 0.93932
[914]	training's auc: 0.985208	valid_1's auc: 0.939312
[915]	training's auc: 0.985233	valid_1's auc: 0.939307
[916]	training's auc: 0.985262	valid_1's auc: 0.939267
[917]	training's auc: 0.985284	valid_1's auc: 0.939256
[918]	training's auc: 0.985302	valid_1's auc: 0.939268
[919]	training's auc: 0.985316	valid_1's auc: 0.939257
[920]	training's auc: 0.985333	valid_1's auc: 0.939248
[921]	training's auc: 0.985347	valid_1's auc: 0.939261
Early stopping, best iteration is:
[821]	training's auc: 0.982895	valid_1's auc: 0.939607
[1]	training's auc: 0.898796	val

[151]	training's auc: 0.95139	valid_1's auc: 0.922031
[152]	training's auc: 0.95144	valid_1's auc: 0.92195
[153]	training's auc: 0.951517	valid_1's auc: 0.921965
[154]	training's auc: 0.951578	valid_1's auc: 0.921968
[155]	training's auc: 0.951625	valid_1's auc: 0.921867
[156]	training's auc: 0.951716	valid_1's auc: 0.921942
[157]	training's auc: 0.951853	valid_1's auc: 0.92201
[158]	training's auc: 0.951895	valid_1's auc: 0.921936
[159]	training's auc: 0.951975	valid_1's auc: 0.921982
[160]	training's auc: 0.952038	valid_1's auc: 0.922002
[161]	training's auc: 0.952093	valid_1's auc: 0.922015
[162]	training's auc: 0.952191	valid_1's auc: 0.922
[163]	training's auc: 0.952248	valid_1's auc: 0.921932
[164]	training's auc: 0.95231	valid_1's auc: 0.921898
[165]	training's auc: 0.952398	valid_1's auc: 0.921982
[166]	training's auc: 0.952443	valid_1's auc: 0.922012
[167]	training's auc: 0.952503	valid_1's auc: 0.921979
[168]	training's auc: 0.952611	valid_1's auc: 0.921992
[169]	training's a

[307]	training's auc: 0.962888	valid_1's auc: 0.923771
[308]	training's auc: 0.962941	valid_1's auc: 0.923784
[309]	training's auc: 0.962991	valid_1's auc: 0.923885
[310]	training's auc: 0.963032	valid_1's auc: 0.923813
[311]	training's auc: 0.96311	valid_1's auc: 0.923802
[312]	training's auc: 0.963181	valid_1's auc: 0.923738
[313]	training's auc: 0.963244	valid_1's auc: 0.923783
[314]	training's auc: 0.963344	valid_1's auc: 0.923884
[315]	training's auc: 0.963413	valid_1's auc: 0.923861
[316]	training's auc: 0.963464	valid_1's auc: 0.923843
[317]	training's auc: 0.963532	valid_1's auc: 0.923935
[318]	training's auc: 0.963591	valid_1's auc: 0.923945
[319]	training's auc: 0.963675	valid_1's auc: 0.923931
[320]	training's auc: 0.963731	valid_1's auc: 0.923902
[321]	training's auc: 0.963775	valid_1's auc: 0.923875
[322]	training's auc: 0.963842	valid_1's auc: 0.923924
[323]	training's auc: 0.963901	valid_1's auc: 0.923957
[324]	training's auc: 0.963965	valid_1's auc: 0.924003
[325]	train

[640]	training's auc: 0.978337	valid_1's auc: 0.924881
[641]	training's auc: 0.978364	valid_1's auc: 0.924888
[642]	training's auc: 0.978398	valid_1's auc: 0.924862
[643]	training's auc: 0.978422	valid_1's auc: 0.924869
[644]	training's auc: 0.978452	valid_1's auc: 0.924826
[645]	training's auc: 0.978475	valid_1's auc: 0.924803
[646]	training's auc: 0.978526	valid_1's auc: 0.924874
[647]	training's auc: 0.978565	valid_1's auc: 0.924862
[648]	training's auc: 0.978605	valid_1's auc: 0.924844
[649]	training's auc: 0.978636	valid_1's auc: 0.924794
[650]	training's auc: 0.978665	valid_1's auc: 0.92476
[651]	training's auc: 0.978701	valid_1's auc: 0.924756
[652]	training's auc: 0.978736	valid_1's auc: 0.924756
[653]	training's auc: 0.978792	valid_1's auc: 0.924795
[654]	training's auc: 0.978829	valid_1's auc: 0.92477
[655]	training's auc: 0.97887	valid_1's auc: 0.924798
[656]	training's auc: 0.978894	valid_1's auc: 0.924799
[657]	training's auc: 0.978915	valid_1's auc: 0.92478
[658]	training

[792]	training's auc: 0.983106	valid_1's auc: 0.924798
[793]	training's auc: 0.983143	valid_1's auc: 0.924851
[794]	training's auc: 0.983187	valid_1's auc: 0.924868
[795]	training's auc: 0.983229	valid_1's auc: 0.924895
[796]	training's auc: 0.983246	valid_1's auc: 0.924902
[797]	training's auc: 0.983276	valid_1's auc: 0.924879
[798]	training's auc: 0.983297	valid_1's auc: 0.924911
[799]	training's auc: 0.983311	valid_1's auc: 0.924928
[800]	training's auc: 0.983341	valid_1's auc: 0.924955
[801]	training's auc: 0.983374	valid_1's auc: 0.924973
[802]	training's auc: 0.983401	valid_1's auc: 0.924924
[803]	training's auc: 0.983437	valid_1's auc: 0.924896
[804]	training's auc: 0.983446	valid_1's auc: 0.924827
[805]	training's auc: 0.983473	valid_1's auc: 0.924827
[806]	training's auc: 0.983499	valid_1's auc: 0.924789
[807]	training's auc: 0.983522	valid_1's auc: 0.924784
[808]	training's auc: 0.98355	valid_1's auc: 0.924796
[809]	training's auc: 0.983569	valid_1's auc: 0.924766
[810]	train